In [1]:
### Standard Libs ###
from collections import defaultdict
from numpy.random import default_rng
import numpy as np
import networkx as nx
from itertools import chain
from tqdm import tqdm

### External Definitions ###
from zone_references import initial_districts
from disease_states import states_dict
from patient_evolution import susceptible_to_exposed, change_state
from actions import city_restrictions

In [2]:
from numpy.random import default_rng

In [3]:
rng = default_rng(42)

In [4]:
g_pickle = '../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle'

In [5]:
G = nx.read_gpickle(g_pickle)

In [6]:
unique_types = np.unique([data['edge_type'] for x,y, data in G.edges(data=True)])
adj_list = {k: defaultdict(list) for k in unique_types}

In [7]:
for node, neighbors in G.adjacency():
    for neighbor, relations in neighbors.items():
        for i, relation_dict in relations.items():
            relation = relation_dict['edge_type']
            adj_list[relation][node].append(neighbor)

In [8]:
currently_infected = rng.choice(G.nodes, size=10000)

In [9]:
contacts = {relation: np.concatenate([c for n, c in neighbors.items() if n in currently_infected])
                for relation, neighbors in adj_list.items()}

In [10]:
prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

In [11]:
restrictions = {k: 0 for k in p_r.keys()}

In [35]:
exposed = []

for rel, prob in p_r.items():
    mask = rng.random(size=len(contacts[rel])) < p_r[rel] * (1-restrictions[rel])
    exposed += contacts[rel][mask].tolist()

In [39]:
exposed = np.unique(exposed)